<a href="https://colab.research.google.com/github/Toluwase/Citation-Weights-based-on-Citation-Contexts-Semantic-Similarity/blob/main/biosentvec_cit_weight.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python packages needed before running the Python codes

These packages were installed on a Google colab jupyter notebook. The syntax on unix or linux may be different.

Step 1: Instal wget (not required)

In [ ]:
!pip install wget

Step 2: Install fastText

In [ ]:
!git clone https://github.com/facebookresearch/fastText.git
!cd fastText
!pip install fastText

Step 3: Install SentVec
(Please note: if you install sent2vec using 
```
$!pip install sent2vec
```
then you'll get the wrong package. Please follow the instructions in the official [Github repository](https://github.com/epfml/sent2vec) to install it correctly. Alternatively, you can use the code below. I wasted many hours trying to figure out the problem during installation because of an error: ```
make: *** No targets specified and no makefile found. Stop.
```

In [ ]:
!wget https://github.com/epfml/sent2vec/archive/master.zip
!unzip master.zip
!make
!sudo pip install sent2vec-master/

# The Python script

In [ ]:
import sent2vec
from nltk import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from scipy.spatial import distance

model_path = ('/content/drive/MyDrive/Colab Notebooks/BioSentVec_PubMed_MIMICIII-bigram_d700.bin')
model = sent2vec.Sent2vecModel()
try:
    model.load_model(model_path)
except Exception as e:
    print(e)
print('model successfully loaded')

stop_words = set(stopwords.words('english'))
def preprocess_sentence(text):
    text = text.replace('/', ' / ')
    text = text.replace('.-', ' .- ')
    text = text.replace('.', ' . ')
    text = text.replace('\'', ' \' ')
    text = text.lower()

    tokens = [token for token in word_tokenize(text) if token not in punctuation and token not in stop_words]

    return ' '.join(tokens)

with open("/content/drive/MyDrive/Colab Notebooks/direct_input/direct_input_1.txt") as my_file:
  lines=my_file.readlines()
#print (lines)
s1 = []
s2 = []
score = []
y=0
for everyline in lines:
    if y<len(lines)-2:
        sentenceONE=preprocess_sentence(lines[y])
        sentenceTWO=preprocess_sentence(lines[y+1])
        sentenceONE_vector = model.embed_sentence(sentenceONE)
        sentenceTWO_vector = model.embed_sentence(sentenceTWO)
        cosine_sim = 1 - distance.cosine(sentenceONE_vector, sentenceTWO_vector)
        print(lines[y],"\t", lines[y+1], "\t", 'cosine similarity:', cosine_sim)
        s1.append(lines[y])
        s2.append(lines[y+1])
        score.append(str(cosine_sim))
    y=y+3
    
#with open('output.txt', "w", encoding="utf-8") as w_f:
#    for i in range(len(s1)):
#        w_f.write(s1[i])
#        w_f.write('\t')
#        w_f.write(s2[i])
#        w_f.write('\t')
#        w_f.write(score[i])
#        w_f.write('\n')